In [1]:
import numpy as np
import pandas as pd

from obj.Compos_DF import ComposDF
from obj.Compo_HTML import *
from obj.List import *
from obj.Block import *
from obj.Group import *
import lib.draw as draw
from obj.Page import Page

In [2]:
name = 'data/9'
compos = ComposDF(name + '.json', name + '.png')
img = compos.img.copy()
img_shape = compos.img_shape
img_re = cv2.resize(img, img_shape)

In [3]:
# ***Step 1*** repetitive list recognition
compos.repetitive_group_recognition()    # group_nontext, group_text
check_valid_group_by_interleaving(compos.compos_dataframe)
compos.pair_groups()                     # group_pair, pair_to, group
compos.list_item_partition()             # list_item
# compos.visualize_block('group')
# compos.visualize_block('group_pair')
# compos.visualize_block('list_item')

invalid: t-6


In [4]:
# ***Step 2*** mark compos in same group as a single list, mark compos in same group_pair as a multiple list
lists, non_listed_compos = gather_lists_by_pair_and_group(compos.compos_dataframe[1:])
generate_lists_html_css(lists)
visualize_lists(img_re, lists)

In [ ]:
def segment_block_h(compos_html):
    compos_html.sort(key=lambda x: x.left)
    
    divider = compos_html[0].right
    block_compos = [compos_html[0]]
    blocks = []

    for compo in compos_html[1:]:
        # segment the previous compos as a block
        if divider < compo.left:
            margin = int(compo.left - divider)
            css_name = '#block-' + str(block_id)
            css = CSS(css_name, margin_right=str(int(compo.top - divider)) + 'px', clear='left', border="solid 2px black")
            blocks.append(Block(id=block_id, compos=block_compos, slice_sub_block_direction=next_direction, html_id='block-' + str(block_id), css={css_name: css}))
            
            divider = compo.right
            block_id += 1
            
            block_compos = []
        # extend block
        elif compo.left < divider < compo.right:
            divider = compo.right
        block_compos.append(compo)
    
    if len(blocks) > 0 and len(block_compos) > 1:
        block_id += 1
        css_name = '#block-' + str(block_id)
        css = CSS(css_name, margin_top=str(int(block_compos[0].top - prev_divider)) + 'px', clear='left', border="solid 2px black")
        blocks.append(Block(id=block_id, compos=block_compos, slice_sub_block_direction=next_direction,
                            html_id='block-' + str(block_id), css={css_name: css}))
    return blocks  

In [ ]:
def segment_block_v(compos_html):
    compos_html.sort(key=lambda x: x.top)
    other_compos = compos_html
    
    divider = compos_html[0].top
    block_compos = [compos_html[0]]
    blocks = []
    
    for compo in compos_html[1:]:
        # segment the previous compos as a block
        if divider < compo.top:
            margin = int(compo.top - divider)
            css_name = '#block-' + str(block_id)
            css = CSS('#block-' + str(block_id)
                      , margin_bottom=str(int(compo.top - divider)) + 'px', clear='left', border="solid 2px black")
            blocks.append(Block(id=block_id, compos=block_compos, slice_sub_block_direction=next_direction, html_id='block-' + str(block_id), css={css_name: css}))
            divider = compo.bottom
            block_id += 1
            # remove blocked compos
            other_compos = list(set(other_compos) - set(block_compos))
            
            blks_v = segment_block_h(other_compos)
            if len(blks_v) > 0:
                blocks += blks_v
                break
                
            block_compos = []
        # extend block
        elif compo.top <= divider < compo.bottom:
            divider = compo.bottom
        block_compos.append(compo)

In [11]:
def slice_blocks(compos_html, direction='v'):
    '''
    Vertically or horizontally scan compos
    :param direction: slice vertically or horizontally
    :param compos_html: CompoHTML objects, including elements and lists
    :return blocks: list of [Block objs]
    :return compos_html: list of compos not blocked: list of [CompoHTML objects]
    '''
    blocks = []
    block_compos = []
    non_blocked_compos = compos_html
    global block_id

    divider = -1
    prev_divider = 0
    if direction == 'v':
        # reverse the direction of next slicing
        next_direction = 'h'
        compos_html.sort(key=lambda x: x.top)
        for compo in compos_html:
            # new block
            # if divider is above than this compo's top, then gather the previous block_compos as a block
            if divider < compo.top:
                prev_divider = divider
                divider = compo.bottom

                margin = int(compo.top - prev_divider)
                # gather previous compos in a block
                # a single compo is not be counted as a block
                if len(block_compos) >= 1:
                    block_id += 1
                    css_name = '#block-' + str(block_id)
                    css = CSS(css_name, margin_top=str(margin) + 'px', clear='left', border="solid 2px black")
                    blocks.append(Block(id=block_id, compos=block_compos, slice_sub_block_direction=next_direction,
                                        html_id='block-'+str(block_id), css={css_name: css}))
                    # remove blocked compos
                    non_blocked_compos = list(set(non_blocked_compos) - set(block_compos))
                block_compos = []
                
                board = img_re.copy()
                compo.visualize(img_re, show=True)
                cv2.line(board, (0, divider), (400, divider), (0,255,0), 2)
                cv2.imshow('line', board)
                cv2.waitKey()
                cv2.destroyAllWindows()
            # extend block
            elif compo.top < divider < compo.bottom:
                divider = compo.bottom
            block_compos.append(compo)

        # if there are some sub-blocks, gather the left compos as a block
        if len(blocks) > 0 and len(block_compos) > 1:
            block_id += 1
            css_name = '#block-' + str(block_id)
            css = CSS(css_name, margin_top=str(int(block_compos[0].top - prev_divider)) + 'px', clear='left', border="solid 2px black")
            blocks.append(Block(id=block_id, compos=block_compos, slice_sub_block_direction=next_direction,
                                html_id='block-' + str(block_id), css={css_name: css}))
            # remove blocked compos
            non_blocked_compos = list(set(non_blocked_compos) - set(block_compos))

    elif direction == 'h':
        # reverse the direction of next slicing
        next_direction = 'v'
        compos_html.sort(key=lambda x: x.left)
        for compo in compos_html:
            # new block
            # if divider is lefter than this compo's right, then gather the previous block_compos as a block
            if divider < compo.left:
                prev_divider = divider
                divider = compo.right

                margin = int(compo.left - prev_divider)
                # gather previous compos in a block
                # a single compo is not to be counted as a block
                if len(block_compos) > 1:
                    block_id += 1
                    css_name = '#block-' + str(block_id)
                    css = CSS(css_name, margin_left=str(margin) + 'px', float='left', border="solid 2px black")
                    blocks.append(Block(id=block_id, compos=block_compos, slice_sub_block_direction=next_direction,
                                        html_id='block-' + str(block_id), css={css_name: css}))
                    # remove blocked compos
                    non_blocked_compos = list(set(non_blocked_compos) - set(block_compos))
                block_compos = []
            # extend block
            elif compo.left < divider < compo.right:
                divider = compo.right
            block_compos.append(compo)

        # if there are some sub-blocks, gather the left compos as a block
        if len(blocks) > 0 and len(block_compos) > 1:
            block_id += 1
            css_name = '#block-' + str(block_id)
            css = CSS(css_name, margin_left=str(int(block_compos[0].left - prev_divider)) + 'px', float='left', border="solid 2px black")
            blocks.append(Block(id=block_id, compos=block_compos, slice_sub_block_direction=next_direction,
                                html_id='block-' + str(block_id), css={css_name: css}))
            # remove blocked compos
            non_blocked_compos = list(set(non_blocked_compos) - set(block_compos))

    return blocks, non_blocked_compos

In [12]:
# ***Step 3*** slice compos as blocks
compos_html = [li.list_obj for li in lists] + non_listed_compos
blocks, non_blocked_compos = slice_blocks(compos_html, 'v')
visualize_blocks(blocks, img, img_shape)

In [16]:
visualize_CompoHTMLs(compos_html, img_re)

In [ ]:
visualize_blocks(blocks, img, img_shape)

In [12]:
blocks[4].visualize_sub_blocks_and_compos(img_re, recursive=True)

Num of sub_block:0; Num of element: 3


array([[[ 77,  15,  42],
        [ 81,  17,  45],
        [ 81,  17,  45],
        ...,
        [ 81,  17,  45],
        [ 81,  17,  45],
        [ 77,  15,  42]],

       [[ 77,  16,  42],
        [ 82,  17,  45],
        [ 82,  17,  45],
        ...,
        [ 82,  17,  45],
        [ 82,  17,  45],
        [ 77,  16,  43]],

       [[ 77,  15,  42],
        [ 82,  17,  45],
        [ 82,  17,  45],
        ...,
        [ 82,  17,  45],
        [ 82,  17,  45],
        [ 77,  16,  43]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [13]:
# ***Step 4*** assembly html and css as web page
page = Page()
for block in blocks:
    page.add_compo(block.html_script, block.css_script)
page.export()